<a href="https://colab.research.google.com/github/pauloday/artbot/blob/stable/Artbot_Studio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artbot Studio
by Paul O'Day

>Based on https://colab.research.google.com/drive/1L8oL-vLJXVcRzCFbPwOoMkPKJ8-aYdPN \

This is the runner for Artbot Studio, which will run in a different window. Here's how to run it:

1.   Go to the 'Runtime' menu, and select 'Change runtime type'. Make sure the accelerator is set to 'GPU'

2.   Run the init cell below by pressing the play button. You'll only have to run this once per session, even after you restart the runtime. It'll take a little while, it has to download at least 2GB of data to Google's servers, but the connection is really fast.

3.   After it's done downloading, run the next cell to upload your run file. You'll have to authenticate for Google Drive. Output will be saved in `Gaillery/`. There will also sometimes be a folder called '.ipynb_checkpoints' generated in your drive/gallery folder. This is safe to delete, and may not even be generated most of the time.

4.   Have fun!


In [ ]:
print('Collecting init files...')
!git clone --branch stable https://github.com/pauloday/artbot
!mv artbot/src/* .
print('Initializing...')
!./artbot/init.sh
import sys
sys.path.append('./taming-transformers')

If you want to stop a run, use the "Restart Runtime" option in the Runtime menu. Doing so won't require you to rerun the setup, you can just start a run right away. If a run doesn't exit succesfully (e.g. out of memory error), or you change a file you just ran, you'll need to restart the runtime. If you write a long video, the progress bar may turn red and stop before it's done (or just not work). This doesn't seem to be a problem, it's just a bug in the video writing library I use.

In [ ]:
from os import remove, path, makedirs
from artbot import Artbot
from google.colab import files
from IPython import display
from tqdm.notebook import tqdm
from google.colab import drive
from pathlib import Path
from shutil import copyfile
# comment this line out if you don't want google drive
drive.mount('/content/drive')
gallery = '/content/drive/MyDrive/Gaillery'
#gallery = '/Gaillery'
#@markdown If this is enabled, the file in your gallery folder will be used.
#@markdown The gallery folder is in your Google drive, called 'Gaillery'.
#@markdown If the file doesn't exist, Artbot will create it and exit.
#@markdown If you don't have a 'scratch.yml' file, one will be created with some demos and documentation.
use_gallery = True #@param {type:"boolean"}
image_pane = display.display(display_id=True)
image_panes = {}
status_panes = {}
def pane(device, typ):
    pane_dict = image_panes if typ == 'image' else status_panes
    if not device in pane_dict.keys():
        pane_dict[device] = display.display(display_id=True)
    return pane_dict[device]

def image_writer(img, dev, video=False):
    if not video:
        pane(dev, 'image').update(display.Image(img))

def status_writer(status, dev):
    st_pane = pane(dev, 'status')
    if not status:
        display.clear_output()
    else:
      st_pane.update(display.Markdown(status))

bot = lambda yaml: Artbot(
    yaml,
    image_writer=image_writer,
    status_writer=status_writer,
    gallery=gallery,
    tqdm=tqdm
)

if use_gallery:
    infile = 'Demo.yml' #@param {type:"string"}
    pathstr = f'{gallery}/{infile}'
    if path.exists(pathstr):
        bot = bot(open(pathstr, 'rb').read())
        bot.run()
    elif infile == 'Demo.yml':
        if not path.exists(gallery):
            makedirs(gallery)
        copyfile('artbot/demo.yml', pathstr)
        print(f'Running demo/documentaiton at {pathstr}')
        bot = bot(open(pathstr, 'rb').read())
        bot.run()
    else:
        Path(pathstr).touch()
        print(f'Created empty file at {pathstr}')
else:
    #@markdown If use_gallery isn't checked, it'll ask you to upload a file
    uploaded = files.upload()
    for fn in uploaded.keys():
        bot = bot(uploaded[fn])
        bot.run()

Send any bugs, suggestions, cool arts, or anything else interesting to artbot@pauloday.com.